## Задание 1

Написать функцию генерации платежной матрицы игры заданной размерности M x N. При этом добавить опциональные параметры (в Матлабе - написать несколько разных функций), позволяющих:
* Сделать просто произвольную платежную матрицу из целых чисел M x N
* Создать такую платежную матрицу M x N где есть ровно k<N последовательно доминирующих стратегий (т.е. если удалить доминирующую строчку, то в оставшейся матрице все равно останется доминирующая строчка)
* Создать такую платежную матрицу M x N где есть ровно k<N последовательно доминируемых стратегий (т.е. по методу СДС должно удаляться ровно k строк)


In [1]:
import numpy as np
import random

random.seed(42)

In [2]:
def sort_first_k_rows_items(matrix: np.ndarray, rows_sort_count: int, reverse: bool = False):
    k = 0
    while k < rows_sort_count and k < matrix.shape[0]:
        # Для каждого столбца
        for j in range(matrix.shape[1]):
            max_by_column_row_num = k
            previous_max = matrix[k][j]

            # Находим максимум в столбце, начиная с k-ой строки
            for i in range(k, matrix.shape[0]):
                if reverse:
                    if matrix[i][j] < previous_max:
                        max_by_column_row_num = i
                        previous_max = matrix[i][j]
                else:
                    if matrix[i][j] > previous_max:
                        max_by_column_row_num = i
                        previous_max = matrix[i][j]

            # Меняем местами элемент k-ой строки и максимальный элемент в этом столбце, начиная с k-ой строки
            matrix[max_by_column_row_num][j] = matrix[k][j]
            matrix[k][j] = previous_max
        k += 1

    # Чтобы было не больше K отсортированных записей, нарушаем порядок в остальных строках
    if matrix.shape[0] <= k:
        return True

    # Пока есть лишние стратегии
    for i in range(k, matrix.shape[0]):
        values_need_suffle = True
        for j in range(matrix.shape[1]):
            if reverse:
                if matrix[i][j] != matrix[k:, j].max():
                    values_need_suffle = False
            else:
                 if matrix[i][j] != matrix[k:, j].min():
                    values_need_suffle = False
        if values_need_suffle:
            return False
    return True

def generate_cost_matrix(rows_count: int, columns_count: int, dominant_strategies_count: int = 0,
                        dominated_strategies_count: int = 0) -> np.ndarray:
    result_cost_matrix = np.zeros((rows_count, columns_count))
    
    # заполнили рандомно платежную матрицу
    for i in range(result_cost_matrix.shape[0]):
        for j in range(result_cost_matrix.shape[1]):
            result_cost_matrix[i][j] = random.randint(1, 500)
    
    # отсортируем первые k строк так, чтобы они были доминирующими стратегиями
    valid = sort_first_k_rows_items(result_cost_matrix, dominant_strategies_count)
    if not valid:
        return generate_cost_matrix(rows_count, columns_count, dominant_strategies_count,
                        dominated_strategies_count)

    # отсортируем первые k строк так, чтобы они были доминируемыми стратегиями
    valid = sort_first_k_rows_items(result_cost_matrix, dominated_strategies_count, reverse = True)
    if not valid:
        return generate_cost_matrix(rows_count, columns_count, dominant_strategies_count,
                            dominated_strategies_count)
    return result_cost_matrix    

Проверим работу метода. Создадим матрицу 5 на 3 с 2 доминирующими стратегями:

Видим, что метод отработал верно. Первые две строки - доминирующие стратегии. Проверим генерацию доминируемых стратегий.
Создадим матрицу 5 на 3 с 2 доминируемыми стратегями:

In [3]:
generate_cost_matrix(5, 3, dominated_strategies_count=2)

array([[ 53.,  58.,  13.],
       [115.,  72.,  45.],
       [380., 141., 378.],
       [328., 347., 380.],
       [457., 280., 126.]])

Видим, что метод отработал верно. Первые две строки - доминируемые стратегии.

## Задание 2

Реализовать метод СДС для произвольных платежных матриц (функция должна выводить, сколько раз у нее получилось удалить строчку доминируемой стратегии)

In [4]:
def dominated_strategies_excluding(matrix: np.ndarray) -> (int, np.ndarray):
    deleted_rows_count = 0

    while True:
        # составляем строку из минимумов
        min_values_by_columns = np.array([min(column) for column in matrix.T])
        rows_to_delete = []

        # проверяем, есть ли такая в матрице
        for i in range(matrix.shape[0]):
            all_items_matched = True
            
            for j in range(matrix.shape[1]):
                if matrix[i][j] != min_values_by_columns[j]:
                    all_items_matched = False
                    break
            if all_items_matched:
                rows_to_delete.append(i)
        
        # Если уже все удалили
        if len(rows_to_delete) == 0 or matrix.shape[0] == 1:
            break
        
        # Удаляем строки
        deleted_rows_count += len(rows_to_delete)
        
        for row_num in reversed(rows_to_delete):
            matrix = np.delete(matrix, row_num, 0)

    print("Работа метода завершена, удалено {0} строк".format(deleted_rows_count))
    return (deleted_rows_count, matrix)


Проверим работу метода. Создадим матрицу с 2 доминируемыми стратегями и запустим метод с полученной матрицей

In [ ]:
matrix = generate_cost_matrix(3, 4, 0, 2)
print("Исходная матрица:")
print(matrix)
print("\n")

deleted_rows_count, matrix = dominated_strategies_excluding(matrix)
print("\n")
print("Матрица после работы метода СДС:")
print(matrix)

Видим, что метод реализован верно.

## Задание 3

Проверить правильность выполнения задания 1 путем запуска в цикле 1000 разных матриц чтобы убедиться что у вас не получилось случайно ни в одной из них лишних доминируемых/доминирующих стратегий (должно быть ровно K)

Проверять будем, запуская метод из задания 2 и сравнивая кол-во удаленных строк с кол-вом доминируемых стратегий.

In [ ]:
K = 5
rows_count = 7
columns_count = 4

for i in range(50):
    # Создаем матрицу методом из задания 1
    matrix = generate_cost_matrix(rows_count, columns_count, dominated_strategies_count=K)
    
    # Запускаем на ней метод из задания 2
    deleted_rows_count, a = dominated_strategies_excluding(matrix)
    
    # Если кол-во удаленных строк не равно K, фиксируем ошибку
    if deleted_rows_count != K:
        print("Ошибка! Количесво удаленных строк методом из задания 2 не равно K")
        print(matrix)
        break
    if (i == 999):
        print("Ошибок не обнаружено")